# Reusing Pretrained Layers
*transfer learning*- using an already trained DNN layers except for the top ones. involves finding an existing neural network that accomplishes a similar task to the one you are trying to tackle

let'a look at an example:

In [16]:
# create a sample model
import tensorflow as tf
import numpy as np

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A.keras")

Epoch 1/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 524us/step - accuracy: 0.5273 - loss: 1.4999 - val_accuracy: 0.8002 - val_loss: 0.6796
Epoch 2/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 470us/step - accuracy: 0.8046 - loss: 0.6340 - val_accuracy: 0.8441 - val_loss: 0.4941
Epoch 3/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 423us/step - accuracy: 0.8497 - loss: 0.4824 - val_accuracy: 0.8631 - val_loss: 0.4198
Epoch 4/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 415us/step - accuracy: 0.8668 - loss: 0.4131 - val_accuracy: 0.8716 - val_loss: 0.3800
Epoch 5/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 423us/step - accuracy: 0.8757 - loss: 0.3732 - val_accuracy: 0.8792 - val_loss: 0.3560
Epoch 6/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 419us/step - accuracy: 0.8823 - loss: 0.3479 - val_accuracy: 0.8824 - val_loss: 0.3399
Epoch 7/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 416us/step - accuracy: 0.8864 - loss: 0.3300 - val_accuracy: 0.8857 - val_loss: 0.3280
Epoch 8/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 428us/step - accuracy: 0.8904 -

In [18]:
# let's load the model

modelA = tf.keras.models.load_model("my_model_A.keras")
model_B_on_A = tf.keras.Sequential(modelA.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

                    

from the above, `modelA` and `model_B_on_A` now share some layers. When one trains `model_B_on_A`, it will also affect `modelA`.

to avoid that, we need to clone the model before reusing its layers as follows:

In [19]:
model_A_clone = tf.keras.models.clone_model(modelA)
model_A_clone.set_weights(modelA.get_weights())


`tf.keras.models.clone_model()` only clones the architecture, not the weights. If you don’t copy them manually using set_weights(), they will be initialized randomly when the cloned model is first used.

let's recreate model_B


In [20]:
model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [21]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])


In [27]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                    validation_data=(X_valid_B, y_valid_B))



Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5699 - loss: 1.3129 - val_accuracy: 0.4857 - val_loss: 0.9044
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5382 - loss: 0.7619 - val_accuracy: 0.4936 - val_loss: 0.7378
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5217 - loss: 0.6696 - val_accuracy: 0.5371 - val_loss: 0.7118
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5895 - loss: 0.6552 - val_accuracy: 0.5559 - val_loss: 0.6975
Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6153 - loss: 0.6309 - val_accuracy: 0.6588 - val_loss: 0.6197
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6873 - loss: 0.5585 - val_accuracy: 0.7240 - val_loss: 0.5569
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7497 - loss: 0.4987 - val_accuracy: 0.7725 - val_loss: 0.5064
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8238 - loss: 0.4510 - val_accuracy: 0.8190 - val_loss: 0.4654
Epoch 5/16

In [28]:
# evaluate
model_B_on_A.evaluate(X_test_B, y_test_B)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.9125 - loss: 0.2900


[0.29054635763168335, 0.9120000004768372]